In [1]:
import pandas as pd
import numpy as np

In [2]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import matplotlib.pylab as plt
import pandas as pd, numpy as np
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from hyperopt import hp, tpe, STATUS_OK, Trials
from hyperopt.fmin import fmin

In [3]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import numpy as np

In [6]:
df = pd.read_csv('training.csv')

In [7]:
def ams(s, b):
    return math.sqrt(2 * ((s + b + 10) * math.log(1.0 + s/(b + 10)) - s))

def get_ams_score(W, Y, Y_pred):
    s = W * (Y == 1) * (Y_pred == 1)
    b = W * (Y == 0) * (Y_pred == 1)
    s = np.sum(s)
    b = np.sum(b)
    return ams(s, b)

In [9]:
model1 = xgb.XGBClassifier(n_estimators =400,colsample_bytree=0.9898730781107081,
                           learning_rate =0.75,
                            max_depth = 12,
                            min_child_weight = 8,
                            subsample = 0.7835130878302855 ,
                           gamma = 0.20827377476218553,
                           reg_lambda = 0.12391233401791746)

In [10]:
def preprocess(X, X_test):
    imputer = Imputer(missing_values = -999.0, strategy = 'most_frequent')
    X = imputer.fit_transform(X)
    X_test = imputer.transform(X_test)
    inv_log_cols = (0,1,2,3,4,5,7,8,9,10,12,13,16,19,21,23,26)
    X_inv_log_cols = np.log(1 / (1 + X[:, inv_log_cols]))
    X = np.hstack((X, X_inv_log_cols))
    X_test_inv_log_cols = np.log(1 / (1 + X_test[:, inv_log_cols]))
    X_test = np.hstack((X_test, X_test_inv_log_cols))
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X_test = scaler.transform(X_test)

    return X, X_test

In [46]:
def single_model():
    classifier = xgb.XGBClassifier(n_estimators =400,colsample_bytree=0.7164544506182514,
                           learning_rate =0.6386564262937965,
                            max_depth = 6,
                            min_child_weight = 9,
                            subsample = 0.7423464990361603,
                           gamma = 0.20827377476218553,
                           reg_lambda = 0.3932413651019372)

    return classifier

In [ ]:
'learning_rate': 0.6386564262937965, 'x_colsample_bytree': 0.7164544506182514, 'x_gamma': 0.1257948856159213, 'x_max_depth': 6.0, 'x_min_child': 9.0, 'x_reg_lambda': 0.3932413651019372, 'x_subsample': 0.7423464990361603

In [12]:
def train_and_predict(X, W, Y, X_test):
    X, X_test = preprocess(X, X_test)
    classifier = single_model()
    classifier.fit(X, Y, sample_weight = W)
    Y_pred = classifier.predict_proba(X)[:,1]
    Y_test_pred = classifier.predict_proba(X_test)[:,1]
    signal_threshold = 83
    cut = np.percentile(Y_test_pred, signal_threshold)
    thresholded_Y_pred = Y_pred > cut
    thresholded_Y_test_pred = Y_test_pred > cut
    
    return [Y_test_pred, thresholded_Y_test_pred]

In [48]:
def write_submission(ids_test, Y_test_pred, thresholded_Y_test_pred):
    ids_probs = np.transpose(np.vstack((ids_test, Y_test_pred)))
    ids_probs = np.array(sorted(ids_probs, key = lambda x: -x[1]))
    ids_probs_ranks = np.hstack((
        ids_probs,
        np.arange(1, ids_probs.shape[0]+1).reshape((ids_probs.shape[0], 1))))

    test_ids_map = {}
    for test_id, prob, rank in ids_probs_ranks:
        test_id = int(test_id)
        rank = int(rank)
        test_ids_map[test_id] = rank

    f = open('submission2.csv', 'w')
    writer = csv.writer(f)
    writer.writerow(['EventId', 'RankOrder', 'Class'])
    for i, pred in enumerate(thresholded_Y_test_pred):
        event_id = int(ids_test[i])
        rank = test_ids_map[ids_test[i]]
        klass = pred and 's' or 'b'
        writer.writerow([event_id, rank, klass])
    f.close()

In [14]:
import csv
import math
import os
import random

import numpy as np
from sklearn.model_selection import *
from sklearn.decomposition import *
from sklearn.ensemble import *
from sklearn.feature_selection import *
from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.pipeline import *
from sklearn.svm import *
from sklearn.tree import *

In [56]:

print ('Loading training data.')
data = np.loadtxt('training.csv',delimiter=',',skiprows=1, converters={32: lambda x:int(x=='s'.encode('utf-8'))})

X = data[:,1:31]
Y = data[:,32]
W = data[:,31]

print ('Loading testing data.')
test_data = np.loadtxt('test.csv',delimiter=',', skiprows=1)

ids_test = test_data[:,0]
X_test = test_data[:,1:31]
W = data[:,31]

    

Loading training data.
Loading testing data.


In [16]:
# y_test

In [19]:
X, X_test = preprocess(X, X_test)

D:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test= train_test_split(
        X, Y, test_size=0.33, random_state=42)

In [21]:
np.sum(y_test)

28054.0

In [22]:
len(X_train)

167500

In [23]:
w_train = W[0:167500] 

In [24]:
len(w_train)

167500

In [25]:
len(X_test)

82500

In [26]:
w_test = W[167500:(167500+82500)] 

In [27]:
len(w_train)

167500

In [28]:
len(X_train)/len(y_train
                )

1.0

In [29]:
w_train

array([0.00265331, 2.23358449, 2.34738894, ..., 0.74405625, 0.01863612,
       0.74405625])

In [30]:
    signal_threshold = 83
    cut = np.percentile(y_test, signal_threshold)

    

In [31]:
thresholded_Y_test_pred = []

In [32]:
for i in y_test:
    if i > cut:
        thresholded_Y_test_pred.append(1)
    else:
        thresholded_Y_test_pred.append(0)
    

In [33]:
y_train = y_train.astype(int)

In [34]:
y_test = thresholded_Y_test_pred

In [36]:
np.asarray(thresholded_Y_test_pred )

array([0, 0, 0, ..., 0, 0, 0])

In [37]:
len(w_train)/len(w_test)

2.0303030303030303

In [38]:

def ams(s, b):
    return math.sqrt(2 * ((s + b + 10) * math.log(1.0 + s/(b + 10)) - s))

def get_ams_score(W, Y, Y_pred):
    s = W * (Y == 1) * (Y_pred == 1)
    b = W * (Y == 0) * (Y_pred == 1)
    s = np.sum(s)
    print(np.sum(s))
    b = np.sum(b)
    print(np.sum(b))
    return ams(s, b)

In [39]:
len(X_test)/len(y_test)

1.0

In [40]:
np.sum(y_train)

57613

In [41]:
w_test

array([4.9768662 , 0.8631929 , 4.48227795, ..., 0.01863612, 1.68161144,
       1.87747381])

In [44]:
def objective(space):
    print(space)
    clf = xgb.XGBClassifier(n_estimators = 400,colsample_bytree=space['colsample_bytree'],
                           learning_rate = space['learning_rate'],
                            max_depth = int(space['max_depth']),
                            min_child_weight = space['min_child_weight'],
                            subsample = space['subsample'],
                           gamma = space['gamma'],
                           reg_lambda = space['reg_lambda'],)
    
    eval_set  = [( X_train, y_train),(X_test, y_test)]
    print('1')
    clf.fit(X_train, y_train,
            eval_set=eval_set,sample_weight=w_train, eval_metric="rmse",
            early_stopping_rounds=10,verbose=False)
    print('3')
    pred = clf.predict_proba(X_test)[:,1]
    signal_threshold = 83
    cut = np.percentile(pred, signal_threshold)
    thresholded_Y_test_pred = []
    for i in pred:
        if i > cut:
            thresholded_Y_test_pred.append(1)
        else:
            thresholded_Y_test_pred.append(0)
    
    print(np.sum(thresholded_Y_test_pred))
#     y_test_pred, pred = train_and_predict(X, W, Y, X_test)
#     clf.fit()
#     pred = clf.predict(X_test)
    print('2')
    mae = mean_absolute_error((y_test), (thresholded_Y_test_pred))
#     ams_score = get_ams_score(w_test,y_test, thresholded_Y_test_pred)
    print ("SCORE:", mae)
    #change the metric if you like
    return {'loss':mae, 'status': STATUS_OK }


space ={'max_depth': hp.quniform("x_max_depth", 4, 16, 1),
        'min_child_weight': hp.quniform ('x_min_child', 1, 10, 1),
        'subsample': hp.uniform ('x_subsample', 0.7, 1),
        'gamma' : hp.uniform ('x_gamma', 0.1,0.7),
        'learning_rate': hp.uniform('learning_rate',0.5,0.8),
        'colsample_bytree' : hp.uniform ('x_colsample_bytree', 0.7,1),
        'reg_lambda' : hp.uniform ('x_reg_lambda', 0,1)
    }


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

print(best)

#{'x_colsample_bytree': 0.9859833603957928, 'x_gamma': 0.4881540428925282, 'x_max_depth': 4.0, 'x_min_child': 4.0, 'x_reg_lambda': 0.9983795315029678, 'x_subsample': 0.9547128225167194}

print(best)

{'colsample_bytree': 0.9510186275634118, 'gamma': 0.18680387090667083, 'learning_rate': 0.7760190514338368, 'max_depth': 7.0, 'min_child_weight': 3.0, 'reg_lambda': 0.2260143374973682, 'subsample': 0.746885102953742}
1
3
4429
2
SCORE: 0.05368484848484849
{'colsample_bytree': 0.8885297557806396, 'gamma': 0.36980940844416743, 'learning_rate': 0.6978245494137797, 'max_depth': 13.0, 'min_child_weight': 3.0, 'reg_lambda': 0.5801079375774624, 'subsample': 0.894974056326531}
1
3
14011
2
SCORE: 0.16983030303030303
{'colsample_bytree': 0.893191652048703, 'gamma': 0.6665868045305081, 'learning_rate': 0.5598442346213914, 'max_depth': 13.0, 'min_child_weight': 2.0, 'reg_lambda': 0.6679417184820324, 'subsample': 0.8423929631697443}
1
3
13880
2
SCORE: 0.16824242424242425
{'colsample_bytree': 0.96307631811762, 'gamma': 0.3429963879608512, 'learning_rate': 0.6176082815840426, 'max_depth': 11.0, 'min_child_weight': 4.0, 'reg_lambda': 0.3490315065026057, 'subsample': 0.9492488971963139}
1
3
14023
2
SCOR

In [45]:
print(best)

{'learning_rate': 0.6386564262937965, 'x_colsample_bytree': 0.7164544506182514, 'x_gamma': 0.1257948856159213, 'x_max_depth': 6.0, 'x_min_child': 9.0, 'x_reg_lambda': 0.3932413651019372, 'x_subsample': 0.7423464990361603}


In [58]:
# Train model now. 

Y_test_pred, thresholded_Y_test_pred = train_and_predict(X, W, Y, X_test)

D:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [50]:
thresholded_Y_test_pred.unique()

AttributeError: 'numpy.ndarray' object has no attribute 'unique'

In [59]:
write_submission(ids_test, Y_test_pred, thresholded_Y_test_pred)

In [55]:
len(thresholded_Y_test_pred)

82500

In [ ]:
print("rin")

In [ ]:
list(df)

In [61]:
import numpy as np
import xgboost as xgb
import math

from sklearn.ensemble import GradientBoostingClassifier as GBC
# from sknn import mlp

from matplotlib import pyplot as plt

In [63]:
# Load traning data:
print ('Loading Data')
n_skiprows = 1
data_train = np.loadtxt('training.csv', delimiter=',', skiprows=n_skiprows, converters={32: lambda x:int(x=='s'.encode('utf-8'))})

Loading Data


In [64]:
np.random.seed(42)
r = np.random.rand(data_train.shape[0])

In [69]:

print ('Assigning data to numpy arrays')
cv_ratio = 0.75
Y_train = data_train[:,32][r<cv_ratio] > 0.
X_train = data_train[:,1:31][r<cv_ratio]
W_train = data_train[:,31][r<cv_ratio]
Y_valid = data_train[:,32][r>=cv_ratio] > 0.
X_valid = data_train[:,1:31][r>=cv_ratio]
W_valid = data_train[:,31][r>=cv_ratio]

print ('Training classifier ...')

xgb_model = xgb.XGBClassifier(n_estimators =400,colsample_bytree=0.7164544506182514,
                           learning_rate =0.6386564262937965,
                            max_depth = 6,
                            min_child_weight = 9,
                            subsample = 0.7423464990361603,
                           gamma = 0.20827377476218553,
                           reg_lambda = 0.3932413651019372)
xgb_model.fit(X_train, Y_train, sample_weight=W_train)

prob_predict_train = xgb_model.predict_proba(X_train)[:,1]
prob_predict_valid = xgb_model.predict_proba(X_valid)[:,1]

pcut = np.percentile(prob_predict_train, 85)

Yhat_train = prob_predict_train > pcut
Yhat_valid = prob_predict_valid > pcut

true_positive_train = W_train * (Y_train == 1.0) * (1.0/cv_ratio)
true_negative_train = W_train * (Y_train == 0.0) * (1.0/cv_ratio)
true_positive_valid = W_valid * (Y_valid == 1.0) * (1.0/(1-cv_ratio))
true_negative_valid = W_valid * (Y_valid == 0.0) * (1.0/(1-cv_ratio))

s_train = sum ( true_positive_train * (Yhat_train == 1.0) )
b_train = sum ( true_negative_train * (Yhat_train == 1.0) )
s_valid = sum ( true_positive_valid * (Yhat_valid == 1.0) )
b_valid = sum ( true_negative_valid * (Yhat_valid == 1.0) )
 
print ('Computing AMS score')
def AMSScore(s,b):
    return math.sqrt (2.*( (s + b + 10.) * math.log(1. + s / (b + 10.)) - s))

ams_train = AMSScore(s_train, b_train)
ams_valid = AMSScore(s_valid, b_valid)

print ('AMS of training set (90%): ', ams_train)
print ('AMS of validation set (10%): ', ams_valid)

# Compute Percent Error training and validation set:
print ('Computing Percentage Error')
predict_train = xgb_model.predict(X_train)
predict_valid = xgb_model.predict(X_valid)

epsilon_error = 0.45
if abs(ams_train - ams_valid) > epsilon_error:
    print ('Generalization Error Too Big')


print ('Loading test data')
data_test = np.loadtxt('test.csv', delimiter=',', skiprows=1)
X_test = data_test[:,1:31]
ID_test = list(data_test[:,0])

print ('Making predictions for test data')
prob_predict_test = xgb_model.predict_proba(X_test)[:,1]
Yhat_test = list(prob_predict_test > pcut)
prob_predict_test = list(prob_predict_test)

print ('Organizing the prediction results')
result_list = []
for x in range(len(ID_test)):
	result_list.append([int(ID_test[x]), prob_predict_test[x], 's'*(Yhat_test[x]==1.0)+'b'*(Yhat_test[x]==0.0)])

result_list = sorted(result_list, key=lambda a_entry: a_entry[1])

for y in range(len(result_list)):
    result_list[y][1] = y+1

result_list = sorted(result_list, key=lambda a_entry: a_entry[0])

print ('Writing CSV file for Kaggle Submission')
fcsv = open('higgsml_output2.csv', 'w')
fcsv.write('EventId,RankOrder,Class\n')
for line in result_list:
    the_line = str(line[0]) + ',' + str(line[1]) + ',' + line[2] + '\n'
    fcsv.write(the_line);
fcsv.close()

Assigning data to numpy arrays
Training classifier ...
Computing AMS score
AMS of training set (90%):  14.242369543157219
AMS of validation set (10%):  3.1563058348298925
Computing Percentage Error
Generalization Error Too Big
Loading test data
Making predictions for test data
Organizing the prediction results


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:117: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


Writing CSV file for Kaggle Submission


In [ ]:
df

In [ ]:
help(df.mask)

In [ ]:
    df = df.replace(-999, np.nan)

In [ ]:
df = df.replace(999, np.nan)

In [ ]:
df = df.fillna(df.mean())

In [ ]:
df = pd.get_dummies(df , columns = ['Label'])

In [ ]:
def ams(s, b):
    return math.sqrt(2 * ((s + b + 10) * math.log(1.0 + s/(b + 10)) - s))

def get_ams_score(W, Y, Y_pred):
    s = W * (Y == 1) * (Y_pred == 1)
    b = W * (Y == 0) * (Y_pred == 1)
    s = np.sum(s)
    b = np.sum(b)
    return ams(s, b)

In [ ]:
y = df.Label_b

In [ ]:
X = df.iloc[:,1:32]

In [ ]:
len(list(X))

In [ ]:
len(list(df))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import numpy as np

In [ ]:
def objective(space):
    print(space)
    clf = xgb.XGBClassifier(n_estimators =1000,colsample_bytree=space['colsample_bytree'],
                           learning_rate = space['learning_rate'],
                            max_depth = int(space['max_depth']),
                            min_child_weight = space['min_child_weight'],
                            subsample = space['subsample'],
                           gamma = space['gamma'],
                           reg_lambda = space['reg_lambda'],)

    eval_set  = [( X_train, y_train),(X_test, y_test)]

    clf.fit(X_train, y_train,
            eval_set=eval_set, eval_metric="rmse",
            early_stopping_rounds=10,verbose=False)

    pred = clf.predict(X_test)
    mse_scr = mean_squared_error(y_test, pred)
    print ("SCORE:", np.sqrt(mse_scr))
    #change the metric if you like
    return {'loss':mse_scr, 'status': STATUS_OK }


space ={'max_depth': hp.quniform("x_max_depth", 4, 16, 1),
        'min_child_weight': hp.quniform ('x_min_child', 1, 10, 1),
        'subsample': hp.uniform ('x_subsample', 0.7, 1),
        'gamma' : hp.uniform ('x_gamma', 0.1,0.7),
        'learning_rate': hp.uniform('learning_rate',0.05,0.4),
        'colsample_bytree' : hp.uniform ('x_colsample_bytree', 0.7,1),
        'reg_lambda' : hp.uniform ('x_reg_lambda', 0,1)
    }


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print(best)

#{'x_colsample_bytree': 0.9859833603957928, 'x_gamma': 0.4881540428925282, 'x_max_depth': 4.0, 'x_min_child': 4.0, 'x_reg_lambda': 0.9983795315029678, 'x_subsample': 0.9547128225167194}

print(best)

In [ ]:
print(best)

In [ ]:
model1 = xgb.XGBRegressor(n_estimators =1000,colsample_bytree=0.9898730781107081,
                           learning_rate =0.37726772129255276,
                            max_depth = 8,
                            min_child_weight = 8,
                            subsample = 0.7835130878302855 ,
                           gamma = 0.20827377476218553,
                           reg_lambda = 0.12391233401791746)

In [ ]:
list(X)

In [ ]:
X_new = X.iloc[:,:-1]

In [ ]:
list(X_new)

In [ ]:
weight = X.Weight

In [ ]:
help(model1)

In [ ]:
model1.fit(X_new,y,sample_weight = weight)

In [ ]:
import pickle


In [ ]:
filename = 'finalized_modelhiggs.sav'
pickle.dump(model1, open(filename, 'wb'))

In [ ]:
df2 = pd.read_csv('test.csv')

In [ ]:
df2 = df2.replace(-999, np.nan)

df2 = df2.replace(999, np.nan)

df2 = df2.fillna(df2.mean())



In [ ]:
list(df2)

In [ ]:
X_test = df2.iloc[:,1:33]

In [ ]:
list(X_test)

In [ ]:
pred = model1.predict(X_test)

In [ ]:
pred


In [ ]:
pred

In [ ]:
df_out = pd.DataFrame(pred)a


In [ ]:
df_out

In [ ]:
y.replace(0,-1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=42)

In [ ]:
def objective(space):
    print(space)
    clf = xgb.XGBRegressor(n_estimators =1000,colsample_bytree=space['colsample_bytree'],
                           learning_rate = space['learning_rate'],
                            max_depth = int(space['max_depth']),
                            min_child_weight = space['min_child_weight'],
                            subsample = space['subsample'],
                           gamma = space['gamma'],
                           reg_lambda = space['reg_lambda'],)

    eval_set  = [( X_train, y_train),(X_test, y_test)]

    clf.fit(X_train, y_train,
            eval_set=eval_set, eval_metric="rmse",
            early_stopping_rounds=10,verbose=False)

    pred = clf.predict(X_test)
    mse_scr = mean_squared_error(y_test, pred)
    print ("SCORE:", np.sqrt(mse_scr))
    #change the metric if you like
    return {'loss':mse_scr, 'status': STATUS_OK }


space ={'max_depth': hp.quniform("x_max_depth", 4, 16, 1),
        'min_child_weight': hp.quniform ('x_min_child', 1, 10, 1),
        'subsample': hp.uniform ('x_subsample', 0.7, 1),
        'gamma' : hp.uniform ('x_gamma', 0.1,0.7),
        'learning_rate': hp.uniform('learning_rate',0.05,0.4),
        'colsample_bytree' : hp.uniform ('x_colsample_bytree', 0.7,1),
        'reg_lambda' : hp.uniform ('x_reg_lambda', 0,1)
    }


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print(best)

#{'x_colsample_bytree': 0.9859833603957928, 'x_gamma': 0.4881540428925282, 'x_max_depth': 4.0, 'x_min_child': 4.0, 'x_reg_lambda': 0.9983795315029678, 'x_subsample': 0.9547128225167194}

print(best)

In [ ]:
print(best)